# Path Visibility in Sample Images

In [1]:
import sys
from pathlib import Path
from typing import List

import json
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from topoutils.constants import ASSETS_DIR, PROJECT_DIR
from topoutils.sphere_sampling import get_spherical_coordinates
from topoutils.visibility import Vertex, from_json

sys.path.insert(0, '..')

## Input data

In [2]:
captured_images = ASSETS_DIR.joinpath('capturedImages')
cameras_sfm = ASSETS_DIR.joinpath('cameras', 'cameras.sfm')

In [3]:
n_range = [2,4,8,16]

## Functions

In [4]:
def read_polylines(n, parent_visibility_folder=PROJECT_DIR.joinpath('visibility')):
    
    visibility_folder = parent_visibility_folder.joinpath(f'n_{n}')
    blue_visibility = from_json(visibility_folder.joinpath('BluePolyline.json'))
    red_visibility = from_json(visibility_folder.joinpath('RedPolyline.json'))
    yellow_visibility = from_json(visibility_folder.joinpath('YellowPolyline.json'))
    green_visibility = from_json(visibility_folder.joinpath('GreenPolyline.json'))

    polylines = [
        (green_visibility, (0, 1, 0.5)), 
        (yellow_visibility, (1, 1, 0)),
        (red_visibility, (1, 0, 0)),
        (blue_visibility, (0, 170/255, 1)),
    ]
    return polylines

In [5]:
def create_output_directory(n, parent_output_directory=PROJECT_DIR.joinpath('output')):
    output_directory = parent_output_directory.joinpath(f'n_{n}')
    output_directory.mkdir(exist_ok=True, parents=True)
    return output_directory

In [6]:
def polyline_to_matrix(visibility):
    vertices = [visibility.edges[0].vertex1, visibility.edges[0].vertex2]
    for edge in visibility.edges[1:]:
        if vertices[-1] == edge.vertex1:
            vertices.append(edge.vertex2)
        elif vertices[-1] == edge.vertex2:
            vertices.append(edge.vertex1)
        else:
            raise ValueError(f'Edge {edge} doesn\'t match')
            
    polyline = np.empty(shape=(len(vertices), 3))

    for i, idx in enumerate(vertices):
        polyline[i][0] = visibility.vertices[idx].x
        polyline[i][1] = visibility.vertices[idx].y
        polyline[i][2] = visibility.vertices[idx].z
    return polyline

In [7]:
def get_image_coordinates(polyline, K, M):
    polyline = np.hstack((polyline[:,:3], np.ones((polyline.shape[0], 1))))
    points = np.matmul(K, np.matmul(M, polyline.T))
    return np.divide(points, points[-1,:]).astype(int)

In [8]:
def calculate_visibility(vertices: List[Vertex], eye: List[float]) -> np.array:
    n = int(np.sqrt(len(vertices[0].visibility_grid)))
    u, v = get_spherical_coordinates(n)
    P = np.array([[v.x, v.y, v.z] for v in vertices])
    delta = eye - P
    R = np.sqrt(np.sum(np.square(delta), axis=1))
    polar_angle = (np.arccos(delta[:, 2]/R) % np.pi).reshape(-1, 1)
    azimuthal_angle = (np.arctan2(delta[:, 1], delta[:, 0]) % (2 * np.pi)).reshape(-1, 1)
    azimuthal_idx = np.argmin(np.abs(u - azimuthal_angle), axis=1)
    polar_idx = np.argmin(np.abs(v - polar_angle), axis=1)
    poly_vis_idx = polar_idx*n+azimuthal_idx
    nn_visibility = [vertex.visibility_grid[vis_idx] for vis_idx, vertex in zip(poly_vis_idx, vertices)]
    return nn_visibility >= R

In [9]:
def clip(coords, image_width, image_height):
    return np.maximum(np.minimum(coords, np.array([[image_width], [image_height], [1]])), 0)

In [10]:
def plot_polyline(image_coords, node_visibility, edges, color, image_width, image_height):
    if not np.any(node_visibility):
        return 
    plt.scatter(
        x=image_coords[:, node_visibility][0], 
        y=image_coords[:, node_visibility][1], 
        facecolors=color,
        edgecolors=color
    )
    for edge in edges:
        v1_visible = node_visibility[edge.vertex1]
        v2_visible = node_visibility[edge.vertex2]
        coords = clip(image_coords[:, [edge.vertex1, edge.vertex2]], image_width, image_height)
        if v1_visible and v2_visible:
            plt.plot(coords[0], coords[1], '-', c=color)
        elif v1_visible or v2_visible:
            plt.plot(coords[0], coords[1], '--', c=color)


In [11]:
def is_inside_image(curr_image_coords, image_width, image_height):
    return np.logical_and(
        np.logical_and(
            curr_image_coords[0] >= 0, 
            curr_image_coords[0] <= image_width,
        ),
        np.logical_and(
            curr_image_coords[1] >= 0, 
            curr_image_coords[1] <= image_height
        )
    )

## Camera information

In [12]:
cameras = json.load(cameras_sfm.open('r'))

In [13]:
cameras.keys()

dict_keys(['version', 'featuresFolders', 'matchesFolders', 'views', 'intrinsics', 'poses'])

In [14]:
intrinsic = cameras['intrinsics'][0]

In [15]:
K = np.array([
    [float(intrinsic["pxFocalLength"]), 0, float(intrinsic["principalPoint"][0]), 0],
    [0, float(intrinsic["pxFocalLength"]), float(intrinsic["principalPoint"][1]), 0],
    [0, 0, 1, 0]
])

In [16]:
views = {view['poseId'] : {
    'imgName': view['path'][view['path'].rfind('/')+1:].upper(),
    'width': int(view['width']),
    'height': int(view['height'])
} for view in cameras['views']}    

In [17]:
for n in n_range:
    polylines = read_polylines(n)
    output_directory = create_output_directory(n)
    
    for pose_obj in tqdm(cameras['poses'], desc=f'n={n}'):
        
        pose = pose_obj['pose']['transform']
        view = views[pose_obj['poseId']]
        img_name = view['imgName']
        im = plt.imread(captured_images.joinpath(img_name))
        fig, ax = plt.subplots(figsize=(16,12))
        implot = ax.imshow(im)
        plt.axis('off')

        R = np.array([float(x) for x in pose["rotation"]]).reshape((3,3), order='F')
        C = np.array([[float(x)] for x in pose["center"]])
        T = - np.matmul(R, C)
        M = np.vstack((np.hstack((R, T)), np.array([0, 0, 0, 1])))
        eye = [float(x) for x in pose["center"]]

        image_width, image_height = view['width'], view['height']

        for curr_polyline, curr_color in polylines:
            polyline_matrix = polyline_to_matrix(curr_polyline)
            curr_image_coords = get_image_coordinates(polyline_matrix, K, M)
            curr_is_visible = np.logical_and(
                calculate_visibility(curr_polyline.vertices, eye),
                is_inside_image(curr_image_coords, image_width, image_height)
            )
            plot_polyline(curr_image_coords, curr_is_visible, curr_polyline.edges, curr_color, image_width, image_height)
        plt.savefig(output_directory.joinpath(img_name), bbox_inches='tight')
        plt.close()

n=16: 100%|████████████████████████████████████████████████████████████████████████████| 77/77 [02:22<00:00,  1.85s/it]
